In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from re import sub

In [2]:
url = "https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&start=1&ref_=adv_nxt"
response = requests.get(url)
soup = BeautifulSoup(response.content,"html.parser")

In [3]:
movie_data = soup.find_all("div",attrs={'class' : 'lister-item mode-advanced'})

In [4]:
movie_data[0]

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt0111161"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt0111161/"> <img alt="The Shawshank Redemption" class="loadlate" data-tconst="tt0111161" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BNDE3ODcxYzMtY2YzZC00NmNlLWJiNDMtZDViZWM2MzIxZDYwXkEyXkFqcGdeQXVyNjAwNDUxODI@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt0111161/">The Shawshank Redemption</a>
<span class="lister-item-year text-muted unbold">(1994)</span>
</h3>
<p class="text-muted">
<span class="certificate">A</span>
<span class="ghost">|</span>
<span class="runtime">142 min</span>
<span class="ghost">|</span>
<span class="genre">
Dram

In [5]:
def fetch_cast(i):
    entity = movie_data[i].find_all('p',class_ = '')[0].find_all('a')
    cast = []
    for i,j in enumerate(entity):
        cast.append(entity[i].text)
    cast.pop(0)
    return cast

In [6]:
def fetch_metascore(i):
    try:
        m_score = movie_data[i].find('span',class_ = 'metascore favorable').text
        m_score = (sub("[^0-9]","",m_score))
    except:
        try:
            m_score = movie_data[i].find('span',class_ = 'metascore mixed').text
            m_score = (sub("[^0-9]","",m_score))
        except:
            try:
                m_score = movie_data[i].find('span',class_ = 'metascore unfavorable').text
                m_score = (sub("[^0-9]","",m_score))
            except:
                m_score = (np.nan)
    return m_score

In [7]:
movie_title = []
release_year = []
runtime = []
genre = []
imdb_rating = []
overview = []
director = []
cast = []
votes = []
Gross = []
meta_score = []
for i in range(1,9952,50):
    url = "https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&start={}&ref_=adv_nxt".format(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content,"html.parser")
    movie_data = soup.find_all("div",attrs={'class' : 'lister-item mode-advanced'})
    for i in range(50):
        try:
            movie_title.append(movie_data[i].h3.a.text)
            year = movie_data[i].h3.find('span',class_ = 'lister-item-year text-muted unbold').text
            release_year.append(sub("[^0-9]","",year))
            time = movie_data[i].p.find('span',class_ = 'runtime').text
            runtime.append(sub("[^0-9]","",time))
            gen = movie_data[i].p.find('span',class_ = 'genre').text
            genre.append(sub('[^A-Za-z0-9 ]',"",gen).strip())
            imdb_rating.append(movie_data[i].find('span',class_ = 'rating-rating').find('span',class_ = 'value').text)
            info = movie_data[i].find_all('p',class_ = 'text-muted')[1].text
            overview.append(sub("[^a-zA-Z0-9 ]","",info))
            director.append(movie_data[i].find('p',class_ = '').a.text)
            cast.append(fetch_cast(i))
            votes.append(movie_data[i].find('p',class_ = 'sort-num_votes-visible').find_all('span')[1].text)
            try:
                Gross.append(movie_data[i].find('p',class_ = 'sort-num_votes-visible').find_all('span',attrs={'data-value': True})[1]['data-value'])
            except:
                Gross.append(np.nan)
            meta_score.append(fetch_metascore(i))
        except:
            pass

In [8]:
cols = ['movie_title','genre','overview','director','cast','runtime','imdb_rating','meta_score','votes','Gross','release_year']

In [9]:
df = pd.DataFrame(list(zip(movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year)),columns=cols)

In [10]:
df.isnull().sum()

movie_title        0
genre              0
overview           0
director           0
cast               0
runtime            0
imdb_rating        0
meta_score      2015
votes              0
Gross           2872
release_year       0
dtype: int64

In [11]:
df.head()

,movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year
0,The Shawshank Redemption,Drama,Over the course of several years two convicts ...,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",142,9.3,82,"2,775,540","28,341,469",1994
1,The Godfather,Crime Drama,Don Vito Corleone head of a mafia family decid...,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K...",175,9.2,100,"1,932,216","134,966,411",1972
2,Ramayana: The Legend of Prince Rama,Animation Action Adventure,An anime adaptation of the Hindu epic the Rama...,Ram Mohan,"[Yûgô Sakô, Koichi Saski, Arun Govil, Nikhil K...",135,9.2,NaN,"12,408",NaN,1993
3,Hababam Sinifi,Comedy Drama,Lazy uneducated students share a very close bo...,Ertem Egilmez,"[Kemal Sunal, Münir Özkul, Halit Akçatepe, Tar...",87,9.2,NaN,"41,977",NaN,1975
4,DAMaN,Adventure Drama,The film is set in 2015 Sid is a young doctor ...,Lenka Debiprasad,"[Vishal Mourya, Karan Kandhapan, Babushan Moha...",121,9.1,NaN,"13,360",NaN,2022


In [12]:
df.tail()

,movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year
9994,Boxing Helena,Drama Mystery Romance,A surgeon becomes obsessed with the seductive ...,Jennifer Lynch,"[Julian Sands, Sherilyn Fenn, Bill Paxton, Kur...",107,4.7,26,"12,752","1,796,389",1993
9995,Zoolander 2,Action Adventure Comedy,Derek and Hansel are lured into modelling agai...,Ben Stiller,"[Ben Stiller, Owen Wilson, Penélope Cruz, Will...",101,4.7,34,"73,434","28,848,693",2016
9996,Capone,Biography Crime Drama,After 10 years in prison 47yearold Al Capone s...,Josh Trank,"[Tom Hardy, Linda Cardellini, Matt Dillon, Al ...",103,4.7,46,"22,125",NaN,2020
9997,I Still Know What You Did Last Summer,Horror Mystery Thriller,The murderous fisherman with a hook is back to...,Danny Cannon,"[Jennifer Love Hewitt, Freddie Prinze Jr., Bra...",100,4.7,21,"76,632","40,002,112",1998
9998,Staying Alive,Drama Music Romance,Five years later Tony Maneros Saturday Night F...,Sylvester Stallone,"[John Travolta, Cynthia Rhodes, Finola Hughes,...",93,4.7,23,"16,947","64,892,670",1983


In [13]:
df.shape

(9999, 11)

In [14]:
df.to_csv("F:/ETL/Dataset/IMDB_10000.csv")

In [15]:
url1 =  "https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&after=WzQuNyw3MTE0LCJ0dDAwODYzNjEiLDEwMDAxXQ%3D%3D&ref_=adv_nxt"

In [16]:
url2 = "https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&after=WzQuNiw0MTc4LCJ0dDEwODc0NjEiLDEwMDUxXQ%3D%3D&ref_=adv_nxt"

In [17]:
url3 = "https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&after=WzQuNSwzMDI0LCJ0dDA0MzAzMDQiLDEwMTAxXQ%3D%3D&ref_=adv_nxt"

In [18]:
url_list = [url1,url2,url3]

In [19]:
url_list

['https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&after=WzQuNyw3MTE0LCJ0dDAwODYzNjEiLDEwMDAxXQ%3D%3D&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&after=WzQuNiw0MTc4LCJ0dDEwODc0NjEiLDEwMDUxXQ%3D%3D&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&after=WzQuNSwzMDI0LCJ0dDA0MzAzMDQiLDEwMTAxXQ%3D%3D&ref_=adv_nxt']

In [20]:
for i in url_list:
    url = i
    response = requests.get(url)
    soup = BeautifulSoup(response.content,"html.parser")
    movie_data = soup.find_all("div",attrs={'class' : 'lister-item mode-advanced'})
    for i in range(50):
        try:
            movie_title.append(movie_data[i].h3.a.text)
            year = movie_data[i].h3.find('span',class_ = 'lister-item-year text-muted unbold').text
            release_year.append(sub("[^0-9]","",year))
            time = movie_data[i].p.find('span',class_ = 'runtime').text
            runtime.append(sub("[^0-9]","",time))
            gen = movie_data[i].p.find('span',class_ = 'genre').text
            genre.append(sub('[^A-Za-z0-9 ]',"",gen).strip())
            imdb_rating.append(movie_data[i].find('span',class_ = 'rating-rating').find('span',class_ = 'value').text)
            info = movie_data[i].find_all('p',class_ = 'text-muted')[1].text
            overview.append(sub("[^a-zA-Z0-9 ]","",info))
            director.append(movie_data[i].find('p',class_ = '').a.text)
            cast.append(fetch_cast(i))
            votes.append(movie_data[i].find('p',class_ = 'sort-num_votes-visible').find_all('span')[1].text)
            try:
                Gross.append(movie_data[i].find('p',class_ = 'sort-num_votes-visible').find_all('span',attrs={'data-value': True})[1]['data-value'])
            except:
                Gross.append(np.nan)
            meta_score.append(fetch_metascore(i))
        except:
            pass

In [21]:
final_df = pd.DataFrame(list(zip(movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year)),columns=cols)

In [22]:
final_df.head()

,movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year
0,The Shawshank Redemption,Drama,Over the course of several years two convicts ...,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",142,9.3,82,"2,775,540","28,341,469",1994
1,The Godfather,Crime Drama,Don Vito Corleone head of a mafia family decid...,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K...",175,9.2,100,"1,932,216","134,966,411",1972
2,Ramayana: The Legend of Prince Rama,Animation Action Adventure,An anime adaptation of the Hindu epic the Rama...,Ram Mohan,"[Yûgô Sakô, Koichi Saski, Arun Govil, Nikhil K...",135,9.2,NaN,"12,408",NaN,1993
3,Hababam Sinifi,Comedy Drama,Lazy uneducated students share a very close bo...,Ertem Egilmez,"[Kemal Sunal, Münir Özkul, Halit Akçatepe, Tar...",87,9.2,NaN,"41,977",NaN,1975
4,DAMaN,Adventure Drama,The film is set in 2015 Sid is a young doctor ...,Lenka Debiprasad,"[Vishal Mourya, Karan Kandhapan, Babushan Moha...",121,9.1,NaN,"13,360",NaN,2022


In [23]:
final_df.tail()

,movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year
10143,Escape Plan: The Extractors,Comedy Romance,A teenage girl sets out to give a nebbish clas...,Mark Waters,"[Addison Rae, Tanner Buchanan, Madison Pettis,...",88,4.4,36,"27,062",NaN,2019
10144,He's All That,Action Adventure Comedy,After being recruited by a group of unconventi...,Renny Harlin,"[Pierce Brosnan, Tim Roth, Nick Cannon, Rami J...",94,4.4,25,"11,819",NaN,2021
10145,The Misfits,Comedy Drama Romance,As a new year at school begins Lolas heart is ...,Lisa Azuelos,"[Miley Cyrus, Douglas Booth, Ashley Greene, De...",97,4.4,NaN,"57,364","46,500",2021
10146,LOL,Comedy,Moronic best friends get themselves locked ins...,Jason Bloom,"[Pauly Shore, Stephen Baldwin, William Atherto...",88,4.4,1,"28,808","26,757,000",2012
10147,Bio-Dome,Drama Mystery Thriller,Novelist Catherine Tramell is once again in tr...,Michael Caton-Jones,"[Sharon Stone, David Morrissey, David Thewlis,...",114,4.4,26,"40,767","5,971,336",1996


In [29]:
final_df.iloc[10099:]

,movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year
10099,Space Jam: A New Legacy,Animation Adventure Comedy,A rogue artificial intelligence kidnaps the so...,Malcolm D. Lee,"[LeBron James, Don Cheadle, Cedric Joe, Khris ...",115,4.5,36,"76,710","70,592,228",2021
10100,Striptease,Comedy Crime Drama,A stripper and single mother gets dragged into...,Andrew Bergman,"[Demi Moore, Burt Reynolds, Armand Assante, Vi...",115,4.5,37,"47,031","33,109,743",1996
10101,Grease 2,Comedy Music Musical,A British student at a 1960s American high sch...,Patricia Birch,"[Michelle Pfeiffer, Maxwell Caulfield, Lorna L...",115,4.5,52,"37,474","15,171,476",1982
10102,The Munsters,Comedy Family Fantasy,Reboot of The Munsters that followed a family ...,Rob Zombie,"[Sheri Moon Zombie, Jeff Daniel Phillips, Dani...",109,4.5,57,"10,409",NaN,2022
10103,Skyline,Action SciFi Thriller,Strange lights descend on the city of Los Ange...,Colin Strause,"[Greg Strause, Eric Balfour, Donald Faison, Sc...",92,4.5,26,"91,223","21,371,425",2010
10104,The Island of Dr. Moreau,Horror SciFi Thriller,After being rescued and brought to an island a...,John Frankenheimer,"[Richard Stanley, David Thewlis, Marlon Brando...",96,4.5,37,"34,653","27,663,982",1996
10105,The Craft: Legacy,Drama Fantasy Horror,A group of high school students form a coven o...,Zoe Lister-Jones,"[Cailee Spaeny, Zoey Luna, Gideon Adlon, Lovie...",97,4.5,54,"15,071",NaN,2020
10106,Thunder Force,Action Adventure Comedy,In a world where supervillains are commonplace...,Ben Falcone,"[Melissa McCarthy, Octavia Spencer, Jason Bate...",106,4.5,34,"44,119",NaN,2021
10107,Occupation,Action Drama SciFi,A small group of town residents have to band t...,Luke Sparke,"[Dan Ewing, Temuera Morrison, Stephany Jacobse...",119,4.5,45,"16,934",NaN,2018
10108,Ouija,Horror Mystery Thriller,A group of friends must confront their most te...,Stiles White,"[Olivia Cooke, Ana Coto, Daren Kagasoff, Bianc...",89,4.5,38,"55,165","50,856,010",2014


In [63]:
new_title = list(final_df.iloc[10138:].movie_title)
time_tick = list(final_df.iloc[10138:].release_year)

In [64]:
new_title.pop(0)
time_tick.pop(0)

'2014'

In [65]:
new_title.append("Basic Instinct 2")
time_tick.append('2006')

In [66]:
new_title

['Peter Pan & Wendy',
 'Cinderella',
 'Supergirl',
 'Escape Plan: The Extractors',
 "He's All That",
 'The Misfits',
 'LOL',
 'Bio-Dome',
 'Basic Instinct 2',
 'Basic Instinct 2']

In [67]:
time_tick

['2009',
 '2023',
 '2021',
 '1984',
 '2019',
 '2021',
 '2021',
 '2012',
 '1996',
 '2006']

In [59]:
final_df.loc[10138:, "movie_title"] = final_df.loc[10138:, "movie_title"].replace(list(final_df.iloc[10138:]["movie_title"]), new_title)

In [60]:
final_df.iloc[10099:]

,movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year
10099,Space Jam: A New Legacy,Animation Adventure Comedy,A rogue artificial intelligence kidnaps the so...,Malcolm D. Lee,"[LeBron James, Don Cheadle, Cedric Joe, Khris ...",115,4.5,36,"76,710","70,592,228",2021
10100,Striptease,Comedy Crime Drama,A stripper and single mother gets dragged into...,Andrew Bergman,"[Demi Moore, Burt Reynolds, Armand Assante, Vi...",115,4.5,37,"47,031","33,109,743",1996
10101,Grease 2,Comedy Music Musical,A British student at a 1960s American high sch...,Patricia Birch,"[Michelle Pfeiffer, Maxwell Caulfield, Lorna L...",115,4.5,52,"37,474","15,171,476",1982
10102,The Munsters,Comedy Family Fantasy,Reboot of The Munsters that followed a family ...,Rob Zombie,"[Sheri Moon Zombie, Jeff Daniel Phillips, Dani...",109,4.5,57,"10,409",NaN,2022
10103,Skyline,Action SciFi Thriller,Strange lights descend on the city of Los Ange...,Colin Strause,"[Greg Strause, Eric Balfour, Donald Faison, Sc...",92,4.5,26,"91,223","21,371,425",2010
10104,The Island of Dr. Moreau,Horror SciFi Thriller,After being rescued and brought to an island a...,John Frankenheimer,"[Richard Stanley, David Thewlis, Marlon Brando...",96,4.5,37,"34,653","27,663,982",1996
10105,The Craft: Legacy,Drama Fantasy Horror,A group of high school students form a coven o...,Zoe Lister-Jones,"[Cailee Spaeny, Zoey Luna, Gideon Adlon, Lovie...",97,4.5,54,"15,071",NaN,2020
10106,Thunder Force,Action Adventure Comedy,In a world where supervillains are commonplace...,Ben Falcone,"[Melissa McCarthy, Octavia Spencer, Jason Bate...",106,4.5,34,"44,119",NaN,2021
10107,Occupation,Action Drama SciFi,A small group of town residents have to band t...,Luke Sparke,"[Dan Ewing, Temuera Morrison, Stephany Jacobse...",119,4.5,45,"16,934",NaN,2018
10108,Ouija,Horror Mystery Thriller,A group of friends must confront their most te...,Stiles White,"[Olivia Cooke, Ana Coto, Daren Kagasoff, Bianc...",89,4.5,38,"55,165","50,856,010",2014


In [68]:
final_df.loc[10138:, "release_year"] = final_df.loc[10138:, "release_year"].replace(list(final_df.iloc[10138:]["release_year"]), time_tick)

In [69]:
final_df.tail()

,movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year
10143,He's All That,Comedy Romance,A teenage girl sets out to give a nebbish clas...,Mark Waters,"[Addison Rae, Tanner Buchanan, Madison Pettis,...",88,4.4,36,"27,062",NaN,2021
10144,The Misfits,Action Adventure Comedy,After being recruited by a group of unconventi...,Renny Harlin,"[Pierce Brosnan, Tim Roth, Nick Cannon, Rami J...",94,4.4,25,"11,819",NaN,2012
10145,LOL,Comedy Drama Romance,As a new year at school begins Lolas heart is ...,Lisa Azuelos,"[Miley Cyrus, Douglas Booth, Ashley Greene, De...",97,4.4,NaN,"57,364","46,500",2012
10146,Bio-Dome,Comedy,Moronic best friends get themselves locked ins...,Jason Bloom,"[Pauly Shore, Stephen Baldwin, William Atherto...",88,4.4,1,"28,808","26,757,000",1996
10147,Basic Instinct 2,Drama Mystery Thriller,Novelist Catherine Tramell is once again in tr...,Michael Caton-Jones,"[Sharon Stone, David Morrissey, David Thewlis,...",114,4.4,26,"40,767","5,971,336",2006


In [70]:
final_df.to_csv("F:/ETL/Dataset/Final_IMDB.csv")

In [75]:
imdb = pd.read_csv("F:/ETL/Dataset/Final_IMDB.csv",index_col=0)

In [76]:
imdb.head()

,movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year
0,The Shawshank Redemption,Drama,Over the course of several years two convicts ...,Frank Darabont,"['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",142,9.3,82.0,"2,775,540","28,341,469",1994
1,The Godfather,Crime Drama,Don Vito Corleone head of a mafia family decid...,Francis Ford Coppola,"['Marlon Brando', 'Al Pacino', 'James Caan', '...",175,9.2,100.0,"1,932,216","134,966,411",1972
2,Ramayana: The Legend of Prince Rama,Animation Action Adventure,An anime adaptation of the Hindu epic the Rama...,Ram Mohan,"['Yûgô Sakô', 'Koichi Saski', 'Arun Govil', 'N...",135,9.2,NaN,"12,408",NaN,1993
3,Hababam Sinifi,Comedy Drama,Lazy uneducated students share a very close bo...,Ertem Egilmez,"['Kemal Sunal', 'Münir Özkul', 'Halit Akçatepe...",87,9.2,NaN,"41,977",NaN,1975
4,DAMaN,Adventure Drama,The film is set in 2015 Sid is a young doctor ...,Lenka Debiprasad,"['Vishal Mourya', 'Karan Kandhapan', 'Babushan...",121,9.1,NaN,"13,360",NaN,2022


In [77]:
imdb.tail()

,movie_title,genre,overview,director,cast,runtime,imdb_rating,meta_score,votes,Gross,release_year
10143,He's All That,Comedy Romance,A teenage girl sets out to give a nebbish clas...,Mark Waters,"['Addison Rae', 'Tanner Buchanan', 'Madison Pe...",88,4.4,36.0,"27,062",NaN,2021
10144,The Misfits,Action Adventure Comedy,After being recruited by a group of unconventi...,Renny Harlin,"['Pierce Brosnan', 'Tim Roth', 'Nick Cannon', ...",94,4.4,25.0,"11,819",NaN,2012
10145,LOL,Comedy Drama Romance,As a new year at school begins Lolas heart is ...,Lisa Azuelos,"['Miley Cyrus', 'Douglas Booth', 'Ashley Green...",97,4.4,NaN,"57,364","46,500",2012
10146,Bio-Dome,Comedy,Moronic best friends get themselves locked ins...,Jason Bloom,"['Pauly Shore', 'Stephen Baldwin', 'William At...",88,4.4,1.0,"28,808","26,757,000",1996
10147,Basic Instinct 2,Drama Mystery Thriller,Novelist Catherine Tramell is once again in tr...,Michael Caton-Jones,"['Sharon Stone', 'David Morrissey', 'David The...",114,4.4,26.0,"40,767","5,971,336",2006


In [78]:
imdb.shape

(10148, 11)